### Olympics Dataset
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

# Summer  Gold  Silver  Bronze  Total  # Winter  Gold.1  \
Afghanistan        13     0       0       2      2         0       0   
Algeria            12     5       2       8     15         3       0   
Argentina          23    18      24      28     70        18       0   
Armenia             5     1       2       9     12         6       0   
Australasia         2     3       4       5     12         0       0   

             Silver.1  Bronze.1  Total.1  # Games  Gold.2  Silver.2  Bronze.2  \
Afghanistan         0         0        0       13       0         0         2   
Algeria             0         0        0       15       5         2         8   
Argentina           0         0        0       41      18        24        28   
Armenia             0         0        0       11       1         2         9   
Australasia         0         0        0        2       3         4         5   

             Combined total   ID  
Afghanistan               2  AFG  
Algeria                  15  ALG  
Argentina                70  ARG  
Armenia                  12  ARM  
Australasia              12  ANZ

### What is the first country in df?

In [2]:
df.iloc[0]

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Which country has won the most gold medals in summer games?

In [3]:
df[df['Gold'] == df['Gold'].max()].index.values[0]

'United States'

### Which country had the biggest difference between their summer and winter gold medal counts?

In [4]:
max_value = (df['Gold'] - df['Gold.1']).max()
max_value_row = df[df['Gold'] - df['Gold.1'] == max_value].index[0]

### Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? (Only include countries that have won at least 1 gold in both summer and winter.)

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

In [5]:
countries = df[(df['Gold'] >= 1) & (df['Gold.1'] >=1)]
metric = (countries['Gold'] - countries['Gold.1']) / countries['Gold.2']
countries[metric == max(metric)].index.values[0]

'Bulgaria'

### Create a Series called "Points" which is a weighted value where each gold medal (Gold.2) counts for 3 points, silver medals (Silver.2) for 2 points, and bronze medals (Bronze.2) for 1 point with country names as indices. 

In [6]:
Points = 3*df['Gold.2'] + 2*df['Silver.2'] + df['Bronze.2']

## Census data from the [United States Census Bureau](http://www.census.gov). This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

In [7]:
census_df = pd.read_csv('census.csv')
census_df.head()

SUMLEV  REGION  DIVISION  STATE  COUNTY   STNAME         CTYNAME  \
0      40       3         6      1       0  Alabama         Alabama   
1      50       3         6      1       1  Alabama  Autauga County   
2      50       3         6      1       3  Alabama  Baldwin County   
3      50       3         6      1       5  Alabama  Barbour County   
4      50       3         6      1       7  Alabama     Bibb County   

   CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  ...  RDOMESTICMIG2011  \
0        4779736            4780127          4785161  ...          0.002295   
1          54571              54571            54660  ...          7.242091   
2         182265             182265           183193  ...         14.832960   
3          27457              27457            27341  ...         -4.728132   
4          22915              22919            22861  ...         -5.527043   

   RDOMESTICMIG2012  RDOMESTICMIG2013  RDOMESTICMIG2014  RDOMESTICMIG2015  \
0         -0.193196          0.381066          0.582002         -0.467369   
1         -2.915927         -3.012349          2.265971         -2.530799   
2         17.647293         21.845705         19.243287         17.197872   
3         -2.500690         -7.056824         -3.904217        -10.543299   
4         -5.068871         -6.201001         -0.177537          0.177258   

   RNETMIG2011  RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
0     1.030015     0.826644     1.383282     1.724718     0.712594  
1     7.606016    -2.626146    -2.722002     2.592270    -2.187333  
2    15.844176    18.559627    22.727626    20.317142    18.293499  
3    -4.874741    -2.758113    -7.167664    -3.978583   -10.543299  
4    -5.088389    -4.363636    -5.403729     0.754533     1.107861  

[5 rows x 100 columns]

### Which state has the most counties in it?

In [25]:
series = census_df.groupby('STATE')['CTYNAME'].count()
index = series[series == series.max()].index[0]
country = census_df[census_df['STATE'] == index]['STNAME'][:1].values
country[0]

'Texas'

### Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

In [9]:
census_df[census_df['SUMLEV']==50].groupby('STNAME')['CENSUS2010POP'].apply(lambda x: x.nlargest(3).sum()).nlargest(3).index.values.tolist()

['California', 'Texas', 'Illinois']

### Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

In [10]:
d = census_df[census_df.SUMLEV==50].copy()
d['max'] = d[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].max(axis=1)
d['min'] = d[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].min(axis=1)
d['diff'] = d['max'] - d['min']
d[d['diff'] == d['diff'].max()].iloc[0]['CTYNAME']

'Harris County'

### Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

In this datafile, the United States is broken up into four regions using the "REGION" column. 

*This function should return a DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [11]:
census_df[((census_df['REGION'] == 1) | (census_df['REGION'] == 2)) & (census_df['CTYNAME'].str.startswith("Washington")) & (census_df['POPESTIMATE2015'] > census_df['POPESTIMATE2014'])].loc[:,['STNAME','CTYNAME']]

STNAME            CTYNAME
896           Iowa  Washington County
1419     Minnesota  Washington County
2345  Pennsylvania  Washington County
2355  Rhode Island  Washington County
3163     Wisconsin  Washington County